In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# from main import get_vsa, get_similarity, get_test_data
# import matplotlib.pyplot as plt

# device = 'cuda'
# vsa = get_vsa(device)
# v1 = vsa.get_vector([(1,1,5,6),(2,0,2,5),(1,2,0,9)], quantize=True)
# print(get_similarity(v1, vsa.get_vector((2,0,2,3)), True))
# print(get_similarity(v1, vsa.get_vector((2,0,2,5)), True))
# dl = get_test_data(vsa)
# plt.imshow(dl.dataset[226][0])

In [2]:
from tools.model_quantization import *
from datasets import get_test_data
from models.nn_non_decomposed import MultiConceptNonDecomposed
import torch
import os.path

root = "./tests/HARDWARE-1024dim-256fd-3x-3y-7color/algo1"
# pt = os.path.join(root, "model_weights_3objs_128batch_20epoch_300000samples_0.0485loss_10-14-21-53.pt")
pt = "tests/HARDWARE-1024dim-256fd-3x-3y-7color/algo1/model_weights_3objs_128batch_20epoch_300000samples_0.0485loss_10-14-21-53.pt"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = MultiConceptNonDecomposed(dim=1024, device=device)
model.eval()
checkpoint = torch.load(pt, map_location=device)
model.load_state_dict(checkpoint)

dl = get_test_data(root, shuffle=True, num_samples=300, max_num_objects=3)
test(model, dl, max_iter=100)


test 1 obj 100 dataset exists, loading...
test 2 obj 100 dataset exists, loading...
test 3 obj 100 dataset exists, loading...
tensor([[ 0.9853, -0.9119,  0.9610,  ..., -1.0568,  1.0321, -0.9299]],
       grad_fn=<AddmmBackward0>)
tensor([[ 1., -1.,  1.,  ..., -1.,  1., -1.]])
0.9905401928590076
0.05127018747510075


In [3]:
merge_biases(model)
remove_biases(model)
test(model, dl, max_iter=100)

tensor([[ 0.9411,  0.9920, -1.0112,  ..., -0.9237,  1.0509, -1.0759]],
       grad_fn=<AddmmBackward0>)
tensor([[ 1.,  1., -1.,  ..., -1.,  1., -1.]])
0.9937522541178335
0.037875847927857156


In [ ]:
stop_profiling = register_activation_profiling_hooks(model)
test(model, dl, max_iter=100)
stop_profiling()

In [ ]:
quantize_layer_weights(model)
quantize_activations(model)
quantize_bottleneck_layers(model)
quantize_averaging_layer(model)
quantize_layer_biases(model)

In [ ]:
test(model, dl, max_iter=100, quantized=True)

In [ ]:
for c in model.model.modules():
    print(c)

In [ ]:
import os
import torch
from models.nn_non_decomposed import MultiConceptNonDecomposed
from tools.model_quantization import profile_model
from tools.dump import generate_model_header, generate_actions
from datasets import get_test_data

root = "./tests/HARDWARE-1024dim-256fd-3x-3y-7color/algo1"
pt = os.path.join(root, "model_weights_3objs_128batch_20epoch_300000samples_0.0485loss_10-14-21-53.pt")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = MultiConceptNonDecomposed(dim=1024, device=device)
model.eval()
checkpoint = torch.load(pt, map_location=device)
model.load_state_dict(checkpoint)
dl = get_test_data(root, shuffle=True, num_samples=300, max_num_objects=3)
profile_model(model, dl)
generate_model_header(model, gemmini_dim=16)
generate_actions(model)